In [3]:
# !pip install --no-deps --upgrade "flash-attn>=2.6.3"
!pip install packaging ninja einops

  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
#     "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Gemma2 patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.724 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.3 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [36]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('dataset/parallel-corpus.xlsx')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

from sklearn.model_selection import train_test_split

# Split the data into training+validation and test sets first (e.g., 80% train+validation, 20% test)
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the training+validation set into training and validation sets (e.g., 75% train, 25% validation)
val, test = train_test_split(test, test_size=0.10, random_state=42)  # 0.25 * 0.8 = 0.2 validation

# Now, 'train' is 60%, 'val' is 20%, and 'test' is 20%
print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

Training set size: 24131
Validation set size: 5429
Test set size: 604


In [37]:
import pandas as pd

# Drop rows where the value in 'Column1' is not a string
train = train[train['SENTENCES '].apply(lambda x: isinstance(x, str))]
train.head()

,SENTENCES,MEANING
7132,This place is so underrated in my opinion,یہ جگہ میری رائے میں بہت کم ہے۔
25770,"Nice place to have in SITE AREA,reasonable pri...",SITE AREA میں اچھی جگہ ہے، پرتعیش سروس کے ساتھ...
29908,Their peri peri chicken tastes brilliant but e...,ان کے پیری پیری چکن کا ذائقہ شاندار ہوتا ہے لی...
19449,Best place for nearest area refreshment,قریبی علاقے کی ریفریشمنٹ کے لیے بہترین جگہ
3418,Why have you come late?,آپ دیر سے کیوں آئے؟


In [38]:
len(train)

24093

In [45]:
prompt = """
<start_of_turn>user
Translate the sentence into urdu and only return the translation:

{}<end_of_turn>
<start_of_turn>model{}\n<end_of_turn>
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
#     instructions = "Translate the sentence into urdu and only return the translation"
    inputs       = examples["SENTENCES "]
    outputs      = examples["MEANING"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [46]:
train_dataset

Dataset({
    features: ['SENTENCES ', 'MEANING', '__index_level_0__'],
    num_rows: 24093
})

In [47]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# len(train_dataset["SENTENCES "])

Map:   0%|          | 0/24093 [00:00<?, ? examples/s]

In [48]:
len(train_dataset)

24093

In [51]:
train_dataset["text"][0]

'\n<start_of_turn>user\nTranslate the sentence into urdu and only return the translation:\n\nThis place is so underrated in my opinion<end_of_turn>\n<start_of_turn>modelیہ جگہ میری رائے میں بہت کم ہے۔\n<end_of_turn>\n<eos>'

In [52]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/24093 [00:00<?, ? examples/s]

/workspace/venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [53]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A4000. Max memory = 15.724 GB.
2.57 GB of memory reserved.


In [54]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24,093 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,011
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,5.885200
2,5.822900
3,6.800700
4,5.388800
5,4.097900
6,3.473100
7,3.433900
8,2.714900
9,2.448700
10,2.427300


In [55]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3073.9505 seconds used for training.
51.23 minutes used for training.
Peak reserved memory = 8.666 GB.
Peak reserved memory for training = 6.096 GB.
Peak reserved memory % of max memory = 55.113 %.
Peak reserved memory for training % of max memory = 38.769 %.


In [56]:
model.save_pretrained("models//gemma_2b_en_ur") # Local saving
tokenizer.save_pretrained("models//gemma_2b_en_ur")

('models//gemma_2b_en_ur/tokenizer_config.json',
 'models//gemma_2b_en_ur/special_tokens_map.json',
 'models//gemma_2b_en_ur/tokenizer.model',
 'models//gemma_2b_en_ur/added_tokens.json',
 'models//gemma_2b_en_ur/tokenizer.json')

In [58]:
model.push_to_hub("darthPanda/gemma_2b_en_ur", token = "HF token") # Online saving
tokenizer.push_to_hub("darthPanda/gemma_2b_en_ur", token = "HF token") # Online saving

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/darthPanda/gemma_2b_en_ur


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

# Evaluation

In [59]:
import pandas as pd

test = pd.read_csv("test_data.csv")
test.head()

,SENTENCES,MEANING,en_ur_2b_bl,ur_en_2b_bl
0,Ambiance was amazing and so the staff. Quality...,ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا مع...,فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔ ...,"It was quite surprising, the atmosphere and th..."
1,You have to promise never to tell anyone what ...,تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہ...,آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ...,"I promise that whatever I tell you, I will nev..."
2,Good place with average food quality,کھانے کے اوسط معیار کے ساتھ اچھی جگہ,خوبہ مکمل غذای میان,Good place with average food.
3,And he lifted his father and mother upon the t...,(شہر میں داخل ہونے کے بعد) اس نے اپنے والدین ک...,وہ اپنے پڑھیاں اور مہم کی چھڑی پر چڑھ لیا، اور...,"After entering the city, he picked up his pare..."
4,What a great place for family safe zone,فیملی سیف زون کے لیے کتنی اچھی جگہ ہے۔,یہ ایک عائلی محفوظ گھڑی ہے,"""This is a good place for the family."""


In [60]:
from tqdm import tqdm
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Define the column you want to process and modify
for index, row in tqdm(test.iterrows()):
    input_sentence = row["SENTENCES "]
    
    prompt = f"""
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    {input_sentence}<end_of_turn>
    <start_of_turn>model
    """

    # alpaca_prompt = You MUST copy from above!

    inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    gen_tokens = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128);

    outs = tokenizer.batch_decode(gen_tokens[:, inputs.input_ids.shape[1]:])[0]
    response_only = outs.replace(tokenizer.eos_token, "").replace("\n<end_of_turn>", "")
    
    test.at[index, 'en_ur_2b_ft'] = response_only

test.head()


0it [00:00, ?it/s]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    Ambiance was amazing and so the staff. Quality of food was also good but the rates are quit high. Their beef mince burger was not up to the mark.
Overall a nice place to visit.<end_of_turn>
    <start_of_turn>model
    اچھا ماحول تھا اور عملہ بھی۔ کھانے کا معیار بھی اچھا تھا لیکن قیمتیں کافی زیادہ ہیں۔ ان کا بیف مینس برگر نشان تک نہیں تھا۔ مجموعی طور پر دیکھنے کے لیے ایک اچھی جگہ۔
    <end_of_turn>
<eos>



1it [00:13, 13.99s/it]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    You have to promise never to tell anyone what I'm about to tell you.<end_of_turn>
    <start_of_turn>model
    تمہیں مجھے بتانے سے پہلے کبھی کسی کو وعدہ کرنا پڑے گا۔
    <end_of_turn>
<eos>



2it [00:15,  6.61s/it]

...

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    ”

The owner did not wait for a second invitation<end_of_turn>
    <start_of_turn>model
    ” مالک ایک دوسری دعوت کے لیے انتظار نہیں کیا۔
    <end_of_turn>
<eos>



603it [16:04,  1.51s/it]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    Four boys are making their way along the street.<end_of_turn>
    <start_of_turn>model
    None
<end_of_turn>
<eos>


604it [16:04,  1.60s/it]


,SENTENCES,MEANING,en_ur_2b_bl,ur_en_2b_bl,en_ur_2b_ft
0,Ambiance was amazing and so the staff. Quality...,ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا مع...,فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔ ...,"It was quite surprising, the atmosphere and th...",اچھا ماحول تھا اور عملہ بھی۔ کھانے کا معیار بھ...
1,You have to promise never to tell anyone what ...,تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہ...,آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ...,"I promise that whatever I tell you, I will nev...",تمہیں مجھے بتانے سے پہلے کبھی کسی کو وعدہ کرنا...
2,Good place with average food quality,کھانے کے اوسط معیار کے ساتھ اچھی جگہ,خوبہ مکمل غذای میان,Good place with average food.,اوسط کھانے کے معیار کے ساتھ اچھی جگہ\n <end...
3,And he lifted his father and mother upon the t...,(شہر میں داخل ہونے کے بعد) اس نے اپنے والدین ک...,وہ اپنے پڑھیاں اور مہم کی چھڑی پر چڑھ لیا، اور...,"After entering the city, he picked up his pare...",11|10|اور اُس نے اپنے باپ اور ماں کو تخت پر بٹ...
4,What a great place for family safe zone,فیملی سیف زون کے لیے کتنی اچھی جگہ ہے۔,یہ ایک عائلی محفوظ گھڑی ہے,"""This is a good place for the family.""",None\n <end_of_turn>\n


In [61]:
test.to_csv('test_data.csv', index=False)